In [1]:
import sys
sys.path.append("src")
import os
os.environ['SAVE_MODEL'] = "1"

In [2]:
import libfeatureselection
import libpybiofeature
from libmodel import t3mm

2023-07-25 10:43:13.423426: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-25 10:43:13.425746: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-25 10:43:13.425754: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-25 10:43:13.977459: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-25 10:43:13.977476: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to c

In [3]:
import numpy as np
import pandas as pd
import json
from Bio import SeqIO

In [4]:
import utils
work_Dir = utils.workdir.workdir(os.getcwd(), 4)
work_Dir

'/mnt/md0/Users/georgezhao/Source/TxSEml_Addon'

In [5]:
protype, cter_bool, db_size = 6, False, 'small'
Tx_arg = {
    "type": f'T{protype}',
    'seq_id': os.path.join(work_Dir, *['data', 'db', f'T{protype}', 'seq_id.json']),
    'shufflesplit_index_file': os.path.join(work_Dir, *['data', 'db', f'T{protype}', 'seq_id_shufflesplit.json']),
    'fasta': {
        'cter': cter_bool,
        'ab': {
            'p': "data/db/T6/t_p.fasta",
            'n': "data/db/T6/t_n.fasta"
        },
        'ae': {
            'p': "data/db/T6/v_p.fasta",
            'n': "data/db/T6/v_n.fasta"
        },
    },
    'bliulab': {
        'PC': {
            'ab': {
                'p': "out/T6/data/tmp/PC-t_p.csv",
                'n': "out/T6/data/tmp/PC-t_n.csv",
            },
            'ae': {
                'p': "out/T6/data/tmp/PC-v_p.csv",
                'n': "out/T6/data/tmp/PC-v_n.csv"
            },
        },
        'Topm': {
            'ab': {
                'p': "out/T6/data/tmp/Topm-t_p.json",
                'n': "out/T6/data/tmp/Topm-t_n.json",
            },
            'ae': {
                'p': "out/T6/data/tmp/Topm-v_p.json",
                'n': "out/T6/data/tmp/Topm-v_n.json"
            },
        },
    },
    "sa": {
        'ab': {
            'p': "out/T6/data/bert/t_p_sa_n.pkl",
            'n': "out/T6/data/bert/t_n_sa_n.pkl",
        },
        'ae': {
            'p': "out/T6/data/bert/v_p_sa_n.pkl",
            'n': "out/T6/data/bert/v_n_sa_n.pkl",
        },
    },
    "diso": {
        'ab': {
            'p': "out/T6/data/bert/t_p_diso_n.pkl",
            'n': "out/T6/data/bert/t_n_diso_n.pkl",
        },
        'ae': {
            'p': "out/T6/data/bert/v_p_diso_n.pkl",
            'n': "out/T6/data/bert/v_n_diso_n.pkl",
        },
    },
    'possum': {
        'index': "out/T6/data/possum/possum_index.json",
        'pssm_fdb_pattern': "out/T6/data/possum/{zipid}_pssm_features.zip",
        'pssm_rdb_pattern': "out/T6/data/possum/{zipid}_pssm_files.zip"
    },
    'model': {
        'size': db_size,
        'cter': cter_bool,
        "path_to_save_dir": f"model/T6/"
    }
}

In [6]:
seq_id_dict = {
    "ab": {
        "p": [ seq.id for seq in SeqIO.parse(Tx_arg['fasta']['ab']['p'], "fasta") ],
        "n": [ seq.id for seq in SeqIO.parse(Tx_arg['fasta']['ab']['n'], "fasta") ],
    },
    "ae": {
        "p": [ seq.id for seq in SeqIO.parse(Tx_arg['fasta']['ae']['p'], "fasta") ],
        "n": [ seq.id for seq in SeqIO.parse(Tx_arg['fasta']['ae']['n'], "fasta") ],
    },
}

In [7]:
feature_data_set = []

In [8]:
feature_data_set.append({
    "name": "AAC",
    "ab_p": libpybiofeature.featurebuilder.build_acc_feature(
        path_to_fasta=Tx_arg['fasta']['ab']['p'],
        seq_id_list=seq_id_dict['ab']['p'],
        desc='ab_p'
    ),
    "ab_n": libpybiofeature.featurebuilder.build_acc_feature(
        path_to_fasta=Tx_arg['fasta']['ab']['n'],
        seq_id_list=seq_id_dict['ab']['n'],
        desc='ab_n'
    ),
    "ae_p": libpybiofeature.featurebuilder.build_acc_feature(
        path_to_fasta=Tx_arg['fasta']['ae']['p'],
        seq_id_list=seq_id_dict['ae']['p'],
        desc='ae_p'
    ),
    "ae_n": libpybiofeature.featurebuilder.build_acc_feature(
        path_to_fasta=Tx_arg['fasta']['ae']['n'],
        seq_id_list=seq_id_dict['ae']['n'],
        desc='ae_n'
    )
})

ae_n_AAC: 100%|██████████| 37/37 [00:00<00:00, 104153.86it/s]


In [9]:
feature_data_set.append({
    "name": "DAC",
    "ab_p": libpybiofeature.featurebuilder.build_dac_feature(
        path_to_fasta=Tx_arg['fasta']['ab']['p'],
        seq_id_list=seq_id_dict['ab']['p'],
        desc='ab_p'
    ),
    "ab_n": libpybiofeature.featurebuilder.build_dac_feature(
        path_to_fasta=Tx_arg['fasta']['ab']['n'],
        seq_id_list=seq_id_dict['ab']['n'],
        desc='ab_n'
    ),
    "ae_p": libpybiofeature.featurebuilder.build_dac_feature(
        path_to_fasta=Tx_arg['fasta']['ae']['p'],
        seq_id_list=seq_id_dict['ae']['p'],
        desc='ae_p'
    ),
    "ae_n": libpybiofeature.featurebuilder.build_dac_feature(
        path_to_fasta=Tx_arg['fasta']['ae']['n'],
        seq_id_list=seq_id_dict['ae']['n'],
        desc='ae_n'
    )
})

ae_n_DAC: 100%|██████████| 37/37 [00:00<00:00, 17224.11it/s]


In [10]:
possum_index_dict = None
with open(Tx_arg['possum']['index'], 'r', encoding='UTF-8') as f:
    possum_index_dict = json.load(f)

feature_data_set.append({
    "name": "PSSM_feature",
    "ab_p": libpybiofeature.libdataloader.pssm_tools.get_all_pssm_feature(
        possum_index_list=possum_index_dict['data']['t_p'],
        feature_name_list=['rpm_pssm', 'pssm_ac', 's_fpssm', 'pse_pssm',],
        path_to_fasta=Tx_arg['fasta']['ab']['p'],
        path_to_with_pattern=Tx_arg['possum']['pssm_fdb_pattern']
    ).loc[seq_id_dict['ab']['p'], :],
    "ab_n": libpybiofeature.libdataloader.pssm_tools.get_all_pssm_feature(
        possum_index_list=possum_index_dict['data']['t_n'],
        feature_name_list=['rpm_pssm', 'pssm_ac', 's_fpssm', 'pse_pssm',],
        path_to_fasta=Tx_arg['fasta']['ab']['n'],
        path_to_with_pattern=Tx_arg['possum']['pssm_fdb_pattern']
    ).loc[seq_id_dict['ab']['n'], :],
    "ae_p": libpybiofeature.libdataloader.pssm_tools.get_all_pssm_feature(
        possum_index_list=possum_index_dict['data']['v_p'],
        feature_name_list=['rpm_pssm', 'pssm_ac', 's_fpssm', 'pse_pssm',],
        path_to_fasta=Tx_arg['fasta']['ae']['p'],
        path_to_with_pattern=Tx_arg['possum']['pssm_fdb_pattern']
    ).loc[seq_id_dict['ae']['p'], :],
    "ae_n": libpybiofeature.libdataloader.pssm_tools.get_all_pssm_feature(
        possum_index_list=possum_index_dict['data']['v_n'],
        feature_name_list=['rpm_pssm', 'pssm_ac', 's_fpssm', 'pse_pssm',],
        path_to_fasta=Tx_arg['fasta']['ae']['n'],
        path_to_with_pattern=Tx_arg['possum']['pssm_fdb_pattern']
    ).loc[seq_id_dict['ae']['n'], :],
})

In [11]:
feature_data_set.append({
    "name": "PPT_full",
    "ab_p": libpybiofeature.featurebuilder.build_PPT_feature(
        path_to_fasta=Tx_arg['fasta']['ab']['p'],
        seq_id_list=seq_id_dict['ab']['p'],
        desc='ab_p',
        fulllong=True
    ),
    "ab_n": libpybiofeature.featurebuilder.build_PPT_feature(
        path_to_fasta=Tx_arg['fasta']['ab']['n'],
        seq_id_list=seq_id_dict['ab']['n'],
        desc='ab_n',
        fulllong=True
    ),
    "ae_p": libpybiofeature.featurebuilder.build_PPT_feature(
        path_to_fasta=Tx_arg['fasta']['ae']['p'],
        seq_id_list=seq_id_dict['ae']['p'],
        desc='ae_p',
        fulllong=True
    ),
    "ae_n": libpybiofeature.featurebuilder.build_PPT_feature(
        path_to_fasta=Tx_arg['fasta']['ae']['n'],
        seq_id_list=seq_id_dict['ae']['n'],
        desc='ae_n',
        fulllong=True
    )
})

ae_n_PPT: 100%|██████████| 37/37 [00:00<00:00, 2816.96it/s]


In [12]:
feature_data_set.append({
    "name": "PPT_25",
    "ab_p": libpybiofeature.featurebuilder.build_PPT_feature(
        path_to_fasta=Tx_arg['fasta']['ab']['p'],
        seq_id_list=seq_id_dict['ab']['p'],
        desc='ab_p'
    ),
    "ab_n": libpybiofeature.featurebuilder.build_PPT_feature(
        path_to_fasta=Tx_arg['fasta']['ab']['n'],
        seq_id_list=seq_id_dict['ab']['n'],
        desc='ab_n'
    ),
    "ae_p": libpybiofeature.featurebuilder.build_PPT_feature(
        path_to_fasta=Tx_arg['fasta']['ae']['p'],
        seq_id_list=seq_id_dict['ae']['p'],
        desc='ae_p'
    ),
    "ae_n": libpybiofeature.featurebuilder.build_PPT_feature(
        path_to_fasta=Tx_arg['fasta']['ae']['n'],
        seq_id_list=seq_id_dict['ae']['n'],
        desc='ae_n'
    )
})

ae_n_PPT: 100%|██████████| 37/37 [00:00<00:00, 19983.16it/s]


In [13]:
feature_data_set.append({
    "name": "18-PP",
    "ab_p": libpybiofeature.featurebuilder.build_etpp_feature(
        path_to_fasta=Tx_arg['fasta']['ab']['p'],
        seq_id_list=seq_id_dict['ab']['p'],
        desc='ab_p'
    ),
    "ab_n": libpybiofeature.featurebuilder.build_etpp_feature(
        path_to_fasta=Tx_arg['fasta']['ab']['n'],
        seq_id_list=seq_id_dict['ab']['n'],
        desc='ab_n'
    ),
    "ae_p": libpybiofeature.featurebuilder.build_etpp_feature(
        path_to_fasta=Tx_arg['fasta']['ae']['p'],
        seq_id_list=seq_id_dict['ae']['p'],
        desc='ae_p'
    ),
    "ae_n": libpybiofeature.featurebuilder.build_etpp_feature(
        path_to_fasta=Tx_arg['fasta']['ae']['n'],
        seq_id_list=seq_id_dict['ae']['n'],
        desc='ae_n'
    )
})

ae_n_etpp: 100%|██████████| 37/37 [00:00<00:00, 6182.10it/s]


In [14]:
feature_data_set.append({
    "name": "CTDD",
    "ab_p": libpybiofeature.featurebuilder.build_CTDD_feature(
        path_to_fasta=Tx_arg['fasta']['ab']['p'],
        seq_id_list=seq_id_dict['ab']['p'],
        desc='ab_p'
    ),
    "ab_n": libpybiofeature.featurebuilder.build_CTDD_feature(
        path_to_fasta=Tx_arg['fasta']['ab']['n'],
        seq_id_list=seq_id_dict['ab']['n'],
        desc='ab_n'
    ),
    "ae_p": libpybiofeature.featurebuilder.build_CTDD_feature(
        path_to_fasta=Tx_arg['fasta']['ae']['p'],
        seq_id_list=seq_id_dict['ae']['p'],
        desc='ae_p'
    ),
    "ae_n": libpybiofeature.featurebuilder.build_CTDD_feature(
        path_to_fasta=Tx_arg['fasta']['ae']['n'],
        seq_id_list=seq_id_dict['ae']['n'],
        desc='ae_n'
    )
})

ae_n_CTDD: 100%|██████████| 37/37 [00:00<00:00, 1076.16it/s]


In [15]:
feature_data_set.append({
    "name": "Top_n_gram",
    "ab_p": libpybiofeature.libdataloader.bliulab_new.Topm(
        path_to_Topm=Tx_arg["bliulab"]['Topm']['ab']['p'],
        seq_id_list=seq_id_dict['ab']['p']
    ),
    "ab_n": libpybiofeature.libdataloader.bliulab_new.Topm(
        path_to_Topm=Tx_arg["bliulab"]['Topm']['ab']['n'],
        seq_id_list=seq_id_dict['ab']['n']
    ),
    "ae_p": libpybiofeature.libdataloader.bliulab_new.Topm(
        path_to_Topm=Tx_arg["bliulab"]['Topm']['ae']['p'],
        seq_id_list=seq_id_dict['ae']['p']
    ),
    "ae_n": libpybiofeature.libdataloader.bliulab_new.Topm(
        path_to_Topm=Tx_arg["bliulab"]['Topm']['ae']['n'],
        seq_id_list=seq_id_dict['ae']['n']
    )
})

In [16]:
def read_PC_data(
    path_to_csv: str,
    path_to_fasta: str,
):
    df = pd.read_csv(path_to_csv, index_col=None, header=None)
    df.index = [
        seq.id for seq in SeqIO.parse(path_to_fasta, "fasta")
    ]
    df.columns = list("ACDEFGHIKLMNPQRSTVWY") + [
        str(i) for i in range(1, (df.shape[1] - 20) + 1)
    ]
    return df

feature_data_set.append({
    "name": "PCPseAAC",
    "ab_p": read_PC_data(
        path_to_csv=Tx_arg["bliulab"]['PC']['ab']['p'],
        path_to_fasta=Tx_arg['fasta']['ab']['p'],
    ),
    "ab_n": read_PC_data(
        path_to_csv=Tx_arg["bliulab"]['PC']['ab']['n'],
        path_to_fasta=Tx_arg['fasta']['ab']['n'],
    ),
    "ae_p": read_PC_data(
        path_to_csv=Tx_arg["bliulab"]['PC']['ae']['p'],
        path_to_fasta=Tx_arg['fasta']['ae']['p'],
    ),
    "ae_n": read_PC_data(
        path_to_csv=Tx_arg["bliulab"]['PC']['ae']['n'],
        path_to_fasta=Tx_arg['fasta']['ae']['n'],
    )
})

In [17]:
feature_data_set.append({
    "name": "DISO-100-DC",
    "ab_p": libfeatureselection.feature_loader.load_from_bert_dac(
        path=Tx_arg['diso']['ab']['p'],
        id_list=seq_id_dict['ab']['p'],
        dim=2
    ),
    "ab_n": libfeatureselection.feature_loader.load_from_bert_dac(
        path=Tx_arg['diso']['ab']['n'],
        id_list=seq_id_dict['ab']['n'],
        dim=2
    ),
    "ae_p": libfeatureselection.feature_loader.load_from_bert_dac(
        path=Tx_arg['diso']['ae']['p'],
        id_list=seq_id_dict['ae']['p'],
        dim=2
    ),
    "ae_n": libfeatureselection.feature_loader.load_from_bert_dac(
        path=Tx_arg['diso']['ae']['n'],
        id_list=seq_id_dict['ae']['n'],
        dim=2
    )
})

In [18]:
feature_data_set.append({
    "name": "SA-100-TC",
    "ab_p": libfeatureselection.feature_loader.load_from_bert_tac(
        path=Tx_arg['sa']['ab']['p'],
        id_list=seq_id_dict['ab']['p'],
        dim=2
    ),
    "ab_n": libfeatureselection.feature_loader.load_from_bert_tac(
        path=Tx_arg['sa']['ab']['n'],
        id_list=seq_id_dict['ab']['n'],
        dim=2
    ),
    "ae_p": libfeatureselection.feature_loader.load_from_bert_tac(
        path=Tx_arg['sa']['ae']['p'],
        id_list=seq_id_dict['ae']['p'],
        dim=2
    ),
    "ae_n": libfeatureselection.feature_loader.load_from_bert_tac(
        path=Tx_arg['sa']['ae']['n'],
        id_list=seq_id_dict['ae']['n'],
        dim=2
    )
})

In [19]:
feature_data_set.append({
    "name": "CKSAAP",
    "ab_p": libpybiofeature.featurebuilder.build_CKSAAP_feature(
        path_to_fasta=Tx_arg['fasta']['ab']['p'],
        seq_id_list=seq_id_dict['ab']['p'],
        desc='ab_p'
    ),
    "ab_n": libpybiofeature.featurebuilder.build_CKSAAP_feature(
        path_to_fasta=Tx_arg['fasta']['ab']['n'],
        seq_id_list=seq_id_dict['ab']['n'],
        desc='ab_n'
    ),
    "ae_p": libpybiofeature.featurebuilder.build_CKSAAP_feature(
        path_to_fasta=Tx_arg['fasta']['ae']['p'],
        seq_id_list=seq_id_dict['ae']['p'],
        desc='ae_p'
    ),
    "ae_n": libpybiofeature.featurebuilder.build_CKSAAP_feature(
        path_to_fasta=Tx_arg['fasta']['ae']['n'],
        seq_id_list=seq_id_dict['ae']['n'],
        desc='ae_n'
    )
})

ae_n_CKSAAP: 100%|██████████| 37/37 [00:00<00:00, 2336.06it/s]


In [20]:
def merge_pn_dataset(
    p_f: pd.DataFrame,
    p_l: np.ndarray,
    n_f: pd.DataFrame,
    n_l: np.ndarray
):
    t_f = pd.concat([p_f, n_f])
    t_l = np.concatenate([p_l, n_l])

    return t_f, t_l

# 合并feature_selected成数据集
data_set_split = {
    datatype: pd.concat([
        item[datatype] for item in feature_data_set
    ], axis=1)
    for datatype in ["ab_p", "ab_n", "ae_p", "ae_n"]
}
label_set_split = {
    datatype: np.ones(
        shape=(feature_data_set[0][datatype].shape[0], ))
    for datatype in ["ab_p", "ae_p",]
} | {
    datatype: np.zeros(
        shape=(feature_data_set[0][datatype].shape[0], ))
    for datatype in ["ab_n", "ae_n",]
}

t_f, t_l = merge_pn_dataset(
    p_f=data_set_split["ab_p"],
    p_l=label_set_split["ab_p"],
    n_f=data_set_split["ab_n"],
    n_l=label_set_split["ab_n"],
)
v_f, v_l = merge_pn_dataset(
    p_f=data_set_split["ae_p"],
    p_l=label_set_split["ae_p"],
    n_f=data_set_split["ae_n"],
    n_l=label_set_split["ae_n"],
)

In [21]:
# 丢入model中进行训练
model_path = f"{Tx_arg['model']['path_to_save_dir']}/1698039127349260/model/"
os.makedirs(
    model_path,
    exist_ok=True
)
model_path

'model/T6//1698039127349260/model/'

In [22]:
from libfeatureselection import model, model_space
n_jobs = (
    (os.cpu_count() - 2)
    if "n_jobs" not in os.environ or os.environ['n_jobs'] == "" else
    int(os.environ['n_jobs'])
)
n_jobs

30

In [23]:
search_result_in_a_scheme_df = pd.DataFrame()
for model_index in range(len(model_space.find_space)):
    model_information_summary, searched_result_performance_summary, searched_result_5C_performance_summary = model.MyOptimitzer(
        classifier_name=model_space.find_space[model_index]['name'],
        classifier_class=model_space.find_space[model_index]['class'],
        classifier_param_dict=model_space.find_space[model_index]['param'],
    ).find_best(
        X=t_f.values,
        y=t_l,
        validation=(
            v_f.values,
            v_l
        ),
        search_method=(
            "BayesSearchCV"
            if "Bayes" not in model_space.find_space[model_index]
            or model_space.find_space[model_index]['Bayes'] == True
            else "GridSearchCV"
        ),
        n_jobs=n_jobs
    ).get_summary(
        path_to_dir=model_path
    )

    # 记录结果，插入到 search_result_in_a_scheme_df
    result_series = pd.concat([
        pd.Series(model_information_summary),
        pd.Series(searched_result_performance_summary),
        pd.Series(searched_result_5C_performance_summary),
    ], keys=[
        "Model_Information",
        "Best_Performance",
        "5FoldCV_Performance",
    ])

    result_series.name = model_index

    search_result_in_a_scheme_df = pd.concat([
        search_result_in_a_scheme_df,
        result_series.to_frame().T
    ], axis=0, ignore_index=False)

    search_result_in_a_scheme_df.index = search_result_in_a_scheme_df.index.set_names(
        ["Model_Type",]
    )

    local_xlsx_path = f"{model_path}/searched_result.xlsx"

    # 缓存 search_result_in_a_scheme_df
    search_result_in_a_scheme_df.to_excel(
        local_xlsx_path,
        "AB-model_Predict_non-AE",
        freeze_panes=(2, 1)
    )

/home/georgezhao/.pyvirtualenvs/TxSEml_Backend/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/georgezhao/.pyvirtualenvs/TxSEml_Backend/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/georgezhao/.pyvirtualenvs/TxSEml_Backend/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 219, in __call__
    return self._score(
  File "/home/georgezhao/.pyvirtualenvs/TxSEml_Backend/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 352, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/home/georgezhao/.pyvirtualenvs/TxSEml_Backend/lib/python3.10/site-packages/sklearn/model_selection/_validatio

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

...

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>